In [1]:
import os
import shutil        # Utility functions for copying and archiving files and directory trees
import random        # Random variable generators
import pandas as pd


# Data Preprocessing

In [8]:
# carga del archivo .csv con la metadata
metadata = pd.read_csv("./data/128x128/metadata.csv")
metadata.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,tfrecord,width,height
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0,0,6000,4000
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0,0,6000,4000
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0,6,1872,1053
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0,0,1872,1053
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0,11,6000,4000


In [12]:
# selección del seed para la distribución homogénea de números aleatorios 
seed = 1
random.seed(seed)

# Selección de directorio a los datos
img_dir = "data/128x128/img/"
# Ruta de datos de entrenamiento, testeo y validación
train = "data/train/"
test = "data/test/"
validation = "data/validation/"

In [ ]:
# Cración de las rutas de datos de entrenamiento, testeo y 
# validacion y separacion por diagnostico positivo o negativo
os.makedirs(train+"negative/")
os.makedirs(train+"positive/")
os.makedirs(test+"negative/")
os.makedirs(test+"positive/")
os.makedirs(validation+"negative/")
os.makedirs(validation+"positive/")

In [9]:
# ontencion de los indices de la tabla
metadata.columns.values


array(['image_name', 'patient_id', 'sex', 'age_approx',
       'anatom_site_general_challenge', 'diagnosis', 'benign_malignant',
       'target', 'tfrecord', 'width', 'height'], dtype=object)

In [10]:
# reasignacion del orden de las colunas de la tabla para aislar el input y el 
# target en las primeras posiciones
metadata_index = ['image_name', 'diagnosis', 'patient_id', 'sex', 'age_approx',
       'anatom_site_general_challenge', 'benign_malignant',
       'target', 'tfrecord', 'width', 'height']
metadata = metadata[metadata_index]
metadata.head()

,image_name,diagnosis,patient_id,sex,age_approx,anatom_site_general_challenge,benign_malignant,target,tfrecord,width,height
0,ISIC_2637011,unknown,IP_7279968,male,45.0,head/neck,benign,0,0,6000,4000
1,ISIC_0015719,unknown,IP_3075186,female,45.0,upper extremity,benign,0,0,6000,4000
2,ISIC_0052212,nevus,IP_2842074,female,50.0,lower extremity,benign,0,6,1872,1053
3,ISIC_0068279,unknown,IP_6890425,female,45.0,head/neck,benign,0,0,1872,1053
4,ISIC_0074268,unknown,IP_8723313,female,55.0,upper extremity,benign,0,11,6000,4000


In [ ]:
# exportando el nuevo archido de metadata 
metadata.to_csv("metadata.csv", index=False)

In [11]:
metadata = pd.read_csv("./metadata.csv")
metadata.head()

,image_name,diagnosis,patient_id,sex,age_approx,anatom_site_general_challenge,benign_malignant,target,tfrecord,width,height
0,ISIC_2637011,unknown,IP_7279968,male,45.0,head/neck,benign,0,0,6000,4000
1,ISIC_0015719,unknown,IP_3075186,female,45.0,upper extremity,benign,0,0,6000,4000
2,ISIC_0052212,nevus,IP_2842074,female,50.0,lower extremity,benign,0,6,1872,1053
3,ISIC_0068279,unknown,IP_6890425,female,45.0,head/neck,benign,0,0,1872,1053
4,ISIC_0074268,unknown,IP_8723313,female,55.0,upper extremity,benign,0,11,6000,4000


In [ ]:
# función que nos separara en las carpetas de test, train y validation 
# las imágenes separándolas a su ves por positive o negative    
test_example = train_example = validation_example = 0
for line in open("metadata.csv").readlines()[1:]:
    split_line = line.split(",")
    img_file = split_line[0]
    lession_type = split_line[1]
    random_num = random.random()
    if random_num < 0.8:
        location = train
        train_example += 1
    elif random_num < 0.9:
        location = validation
        validation_example += 1
    else:
        location = test
        test_example += 1
    if lession_type == "melanoma":
        shutil.copy(
            img_dir+img_file+'.jpg',
            location+"positive/"+img_file+'.jpg'
        )
    elif lession_type != "melanoma":
        shutil.copy(
            img_dir+img_file+'.jpg',
            location+"negative/"+img_file+'.jpg'
        )
# impresión del numero de elementos en train, test y validation
print(f"Number of training examples {train_example}")
print(f"Number of test examples {test_example}")
print(f"Number of validation examples {validation_example}")


FileNotFoundError: [Errno 2] No such file or directory: 'data/128x128/train/ISIC_2637011.jpg'